In [ ]:
import torch
import numpy as np
import mdtraj

import boltzgen.zmatrix as zmatrix
import boltzgen.internal as ics
import boltzgen.mixed as mixed

import normflow as nf
from boltzgen.flows import CoordinateTransform
from boltzgen.distributions import Boltzmann, BoltzmannParallel, TransformedBoltzmann, TransformedBoltzmannParallel

from tqdm import tqdm
from matplotlib import pyplot as plt

from autograd import grad
from autograd import numpy as np
from simtk import openmm as mm
from simtk import unit
from simtk.openmm import app
from openmmtools.testsystems import AlanineDipeptideVacuum

# Load the alanine dipeptide trajectory
aldp_traj = mdtraj.load('alanine_dipeptide/trajectory/aldp100000.h5')

In [ ]:
# Set up coordinate transformation

z_matrix = [
    (0, [1, 4, 6]),
    (1, [4, 6, 8]),
    (2, [1, 4, 0]),
    (3, [1, 4, 0]),
    (4, [6, 8, 14]),
    (5, [4, 6, 8]),
    (7, [6, 8, 4]),
    (11, [10, 8, 6]),
    (12, [10, 8, 11]),
    (13, [10, 8, 11]),
    (15, [14, 8, 16]),
    (16, [14, 8, 6]),
    (17, [16, 14, 15]),
    (18, [16, 14, 8]),
    (19, [18, 16, 14]),
    (20, [18, 16, 19]),
    (21, [18, 16, 19])
]

cart_indices = [6, 8, 9, 10, 14]

aldp_traj.center_coordinates()

# superpose on the backbone
ind = aldp_traj.top.select("backbone")

aldp_traj.superpose(aldp_traj, 0, atom_indices=ind, ref_atom_indices=ind)

# Gather the training data into a pytorch Tensor with the right shape
training_data = aldp_traj.xyz
n_atoms = training_data.shape[1]
n_dim = n_atoms * 3
training_data_npy = training_data.reshape(-1, n_dim)
training_data = torch.from_numpy(training_data_npy.astype("float64"))

In [ ]:
# Set up simulation object for energy computation

temperature = 1000

testsystem = AlanineDipeptideVacuum(constraints=None)
implicit_sim = app.Simulation(testsystem.topology,
                              testsystem.system,
                              mm.LangevinIntegrator(temperature * unit.kelvin , 1.0 / unit.picosecond, 1.0 * unit.femtosecond),
                              mm.Platform.getPlatformByName('Reference')#,
                              #{'Precision': 'double'}
                              )
implicit_sim.context.setPositions(testsystem.positions)

In [ ]:
# Set up model

# Define flows
K = 5
torch.manual_seed(0)

# Set prior and q0
p = Boltzmann(implicit_sim.context, temperature, energy_cut=1e2, energy_max=1e20)


transform = CoordinateTransform(training_data, 66, z_matrix, cart_indices)

p_ = TransformedBoltzmannParallel(testsystem, temperature, energy_cut=1e2,
                                  energy_max=1e20, transform=transform)
p__ = BoltzmannParallel(testsystem, temperature, energy_cut=1e2, energy_max=1e20)

latent_size = 60
hidden_units = 128
q0 = nf.distributions.DiagGaussian(latent_size, trainable=False)

b = torch.Tensor([1 if i % 2 == 0 else 0 for i in range(latent_size)])
flows = []
for i in range(K):
    # Add two alternating Real NVP layers, and ActNorm layer, and a MCMC layer
    # Real NVP layers
    s = nf.nets.MLP([latent_size, hidden_units, hidden_units, hidden_units, latent_size])
    t = nf.nets.MLP([latent_size, hidden_units, hidden_units, hidden_units, latent_size])
    flows += [nf.flows.MaskedAffineFlow(b, s, t)]
    s = nf.nets.MLP([latent_size, hidden_units, hidden_units, hidden_units, latent_size])
    t = nf.nets.MLP([latent_size, hidden_units, hidden_units, hidden_units, latent_size])
    flows += [nf.flows.MaskedAffineFlow(1 - b, s, t)]
    # ActNorm
    flows += [nf.flows.ActNorm(latent_size)]
    # MCMC layer
    dist = nf.distributions.LinearInterpolation(p_, q0, (i + 1) / K)
    proposal = nf.distributions.DiagGaussianProposal((latent_size,),
                                                     0.1 * np.ones(latent_size))
    flows += [nf.flows.MetropolisHastings(dist, proposal, 20)]
flows += [transform]

# Construct flow model
nfm = nf.NormalizingFlow(q0=q0, flows=flows, p=p)

# Move model on GPU if available
enable_cuda = False
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')
nfm = nfm.to(device)
nfm = nfm.double()
#nfm = nfm.float()

# Initialize ActNorm
ind = torch.randint(len(training_data), (256, ))
x = training_data[ind, :].double().to(device)
#x = training_data[ind, :].float().to(device)
#x.requires_grad = True
from time import time
t = time()
kld = nfm.forward_kld(x)
print(time() - t)

In [ ]:
print(kld)

In [ ]:
# Train model
batch_size = 256
num_samples = 32
max_iter = 10
trans_iter = 8000
n_data = len(training_data)
eval_rkld = 10


loss_hist = np.array([])
fkld_hist = np.array([])
rkld_hist = np.array([])

optimizer = torch.optim.AdamW(nfm.parameters(), lr=1e-4, weight_decay=1e-5)
for it in tqdm(range(max_iter)):
    #nfm.p.alpha = np.max([0., 1 - it / trans_iter])
    optimizer.zero_grad()
    ind = torch.randint(n_data, (batch_size, ))
    x = training_data[ind, :].double().to(device)
    fkld = nfm.forward_kld(x)
    #rkld = nfm.reverse_kld(num_samples=num_samples)
    loss = fkld #+ rkld
    if not torch.isnan(loss) and loss < 0:
        loss.backward()
        #torch.nn.utils.clip_grad_value_(nfm.parameters(), .01)
        #gradient_norm = torch.nn.utils.clip_grad.clip_grad_norm_(nfm.parameters(), 100.)
        optimizer.step()
    
    loss_hist = np.append(loss_hist, loss.to('cpu').data.numpy())
    fkld_hist = np.append(fkld_hist, fkld.to('cpu').data.numpy())
    #rkld_hist = np.append(rkld_hist, rkld.to('cpu').data.numpy())

In [ ]:
loss_hist[loss_hist > 0] = np.nan
plt.plot(loss_hist)
plt.show()
#plt.plot(fkld_hist)
#plt.show()
#plt.plot(rkld_hist)
#plt.show()

In [ ]:
nfm.eval()

z_np = np.zeros((0, 60))
x_np = np.zeros((0, 66))
log_p_np = np.zeros((0,))
log_q_np = np.zeros((0,))
for i in tqdm(range(10)):
    z, log_q = nfm.sample(1000)
    x_np = np.concatenate((x_np, z.cpu().data.numpy()))
    log_p = nfm.p.log_prob(z)
    z, _ = nfm.flows[-1].inverse(z)
    z_np_ = z.cpu().data.numpy()
    log_p_np_ = log_p.cpu().data.numpy()
    log_q_np_ = log_q.cpu().data.numpy()
    z_np = np.concatenate((z_np, z_np_))
    log_p_np = np.concatenate((log_p_np, log_p_np_))
    log_q_np = np.concatenate((log_q_np, log_q_np_))


z_d = training_data[::10].double().to(device)
#z_d = training_data[::10].float().to(device)
#z_d.requires_grad = True
log_p_d = nfm.p.log_prob(z_d)
z_d, _ = nfm.flows[-1].inverse(z_d)
z_d_np = z_d.cpu().data.numpy()

log_p_d_np = log_p_d.cpu().data.numpy()

In [ ]:
for i in range(60):
    print(i)
    plt.hist(z_d_np[:, i], bins=100, alpha=0.5, label='data', range=[-3.1, 3.1])
    plt.hist(z_np[:, i], bins=100, alpha=0.5, label='samples', range=[-3.1, 3.1])
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
import mdtraj
Z_indices = np.array([[4, 6, 8, 14],
                      [11, 10, 8, 6],
                      [16, 14, 8, 6],
                      [1, 4, 6, 8],
                      [5, 4, 6, 8],
                      [7, 6, 8, 4],
                      [12, 10, 8, 4],
                      [13, 10, 8, 11],
                      [15, 14, 8, 16],
                      [18, 16, 14, 8],
                      [0, 1, 4, 6],
                      [17, 16, 14, 15], 
                      [19, 18, 16, 14],
                      [2, 1, 4, 0],
                      [3, 1, 4, 0],
                      [20, 18, 16, 19],
                      [21, 18, 16, 19]])

training_data_traj = mdtraj.load('/alanine_dipeptide/trajectory/aldp100000.h5')
torsions_train = mdtraj.compute_dihedrals(training_data_traj, Z_indices)

ala2_pdb = mdtraj.load('alanine_dipeptide/alanine-dipeptide.pdb').topology
gen_data_traj = mdtraj.Trajectory(x_np.reshape(-1, 22, 3), ala2_pdb)
torsions_gen = mdtraj.compute_dihedrals(gen_data_traj, Z_indices)

In [ ]:
def periodic_convolution(x, kernel):
    x_padded = np.concatenate([x, x, x])
    y_padded = np.convolve(x_padded, kernel, mode='same')
    return y_padded[x.size:-x.size]

torsion_hists_train = []
torsion_hists_gen = []
xticks = None

for i in range(torsions_train.shape[1]):
    htrain, e = np.histogram(torsions_train[:, i], 50, range=(-np.pi, np.pi), density=True);
    xticks = 0.5 * (e[1:] + e[:-1])
    hgen, _ = np.histogram(torsions_gen[:, i], 50, range=(-np.pi, np.pi), density=True);
    
    htrain = periodic_convolution(htrain, [0.25, 0.5, 1.0, 0.5, 0.25])
    hgen = periodic_convolution(hgen, [0.25, 0.5, 1.0, 0.5, 0.25])
    
    torsion_hists_train.append(htrain)
    torsion_hists_gen.append(hgen)
    
torsions_simple = [4, 5, 6, 7, 8]
torsions_complex = [0, 1, 2, 10, 12]

In [ ]:
#fig, axes = plt.subplots(nrows=2, ncols=5, sharex=True, figsize=(15, 5))
fig, axes = plt.subplots(nrows=1, ncols=5, sharey=True, sharex=True, figsize=(15, 3))
axes = axes.reshape((1, 5))
fig.subplots_adjust(hspace=0.05, wspace=0.15)
#for row, torsion_index in zip([0, 1], [torsions_simple, torsions_complex]):
for row, torsion_index in zip([0], [torsions_complex]):
    for i, ax in enumerate(axes[row]):
        ax.plot(xticks, torsion_hists_train[torsion_index[i]], color='grey', linewidth=5)
        ax.plot(xticks, torsion_hists_gen[torsion_index[i]], color='red', linewidth=3)
        ax.set_yticks([])
        ax.set_xlim(-np.pi, np.pi)
        if row == 0:
            ax.set_ylim(0, 5)
        if row == 1:
            ax.set_ylim(0, 1.5)
        if row == 1:
            ax.set_xticks((-np.pi,0, np.pi))
            ax.set_xticklabels(('$-\pi$', 0,'$\pi$'))
    axes[-1,-1].set_yticks([])
axes[0, 0].set_ylim(0, 1.5)
axes[0, 0].text(0, 1.35, '$\phi$')
axes[0, 1].text(0, 1.35, '$\gamma_1$')
axes[0, 2].text(0, 1.35, '$\psi$')
axes[0, 3].text(0, 1.35, '$\gamma_2$')
axes[0, 4].text(0, 1.35, '$\gamma_3$')
axes[0, 0].set_ylabel('density')
#axes[1, 0].set_ylabel('density')
axes[0, -1].text(-np.pi+0.3, 1.1, 'Target', color='grey')
axes[0, -1].text(-np.pi+0.3, 0.7, 'RNVP + MCMC', color='red')
plt.savefig('torsion_angles.png', dpi=300)

In [ ]:
eps = 1e-5
for i in torsions_complex:
    KL_SNF = np.sum(torsion_hists_train[i] * np.log((torsion_hists_train[i]+eps) / (torsion_hists_gen[i]+eps)))
    print("{:1.2f}".format(KL_SNF))